In [21]:
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
import matplotlib.pyplot as plt
from dw_tap.lom import run_lom
import os
import seaborn as sns
import glob
from dw_tap.data_processing import _LatLon_To_XY, filter_obstacles
import subprocess
import shutil
from dw_tap.obstacles import AllObstacles
from dw_tap.lom import run_lom
import time
import logging
logger = logging.getLogger()

# Hide all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import sys
sys.path.append("../../scripts")
import dw_tap_data 

def test_run_t024():
    index = pd.read_csv(os.path.join(dw_tap_data.path, "01 Bergey Turbine Data/bergey_sites.csv"))
    
    tid = 't024'
    #wind_source = "wtk_led_2018"
    obstacle_mode = "treesasbldgs"
    
    wtk = pd.read_csv(os.path.join(dw_tap_data.path, "01 Bergey Turbine Data/wtk_tp.csv.bz2"))
    
    tmp = wtk[wtk["tid"] == tid].copy().reset_index(drop=True)
    tmp["datetime"] = pd.to_datetime(tmp["packet_date"])
       
    # 1 year of data
    #start = pd.to_datetime('2013-01-01 00:00:00+00:00') 
    #end = pd.to_datetime('2013-12-31 23:59:00+00:00')
    #tmp = tmp[(tmp["datetime"] >= start) & (tmp["datetime"] <= end)].reset_index(drop=True)
    
    atmospheric_input = tmp
    
    #print("atmospheric_input:")
    #display(atmospheric_input)
    
    # Prepare obstacles input
    
    obs = AllObstacles(data_dir=dw_tap_data.path, types=["bergey"], debug=False)
    obs_subset = obs.get("bergey", tid, obstacle_mode)
    print("obs_subset:", obs_subset)
    #obs_subset
    
    # Run ANL's LOM

    row = index[index["APRS ID"] == tid].iloc[0]
    lat = row["Latitude"]
    lon = row["Longitude"]
    z_turbine = row["Hub Height (m)"]
    xy_turbine = [np.array([lon, lat])]
    print(lat, lon, z_turbine)
    
    t_lom_start = time.time()
    predictions_df = run_lom(atmospheric_input, obs_subset, xy_turbine, z_turbine, check_distance=True)
    t_lom = time.time() - t_lom_start    
    logger.info("'Reporting the runtime for PILOWF: %.3f (s)' % t_lom")
    
    display(predictions_df.sort_values("ws-adjusted"))
 
    pmin = predictions_df["ws-adjusted"].min() 
    pavg = predictions_df["ws-adjusted"].mean() 
    pmax = predictions_df["ws-adjusted"].max() 
    assert pmin >= 0, "Testing PILOWF: min ws-adjusted should be non-negative (observed: %f)." % pmin
    assert pavg >= 0, "Testing PILOWF: avg ws-adjusted should be non-negative (observed: %f)." % pavg
    assert pmax <= 150, "Testing PILOWF: realistic max ws-adjusted should be <= 150 (observed: %f)." % pmax
    
test_run_t024()

obs_subset:        height                                           geometry feature_type  \
0    6.360962  POLYGON ((-119.82480 39.33605, -119.82500 39.3...     building   
1    4.615967  POLYGON ((-119.82413 39.32681, -119.82422 39.3...     building   
2    3.846680  POLYGON ((-119.81708 39.32649, -119.81715 39.3...     building   
3    7.563354  POLYGON ((-119.81710 39.32658, -119.81697 39.3...     building   
4    5.599060  POLYGON ((-119.82232 39.32961, -119.82245 39.3...     building   
..        ...                                                ...          ...   
89  11.475830  POLYGON ((-119.82098 39.33158, -119.82101 39.3...         tree   
90   6.375244  POLYGON ((-119.82090 39.33064, -119.82092 39.3...         tree   
91   3.312744  POLYGON ((-119.81928 39.33100, -119.81930 39.3...         tree   
92  15.315430  POLYGON ((-119.81914 39.33076, -119.81916 39.3...         tree   
93  18.640991  POLYGON ((-119.81903 39.33130, -119.81906 39.3...         tree   

     tid  
0   

/Users/dduplyak/anaconda3/envs/dw-tap/lib/python3.8/site-packages/dw_tap-0.2-py3.8.egg/dw_tap/lom.py:149: RuntimeWarning: overflow encountered in exp
  factors=(1. / (1. + np.exp(steepness*(plot_test_data[:,3] - 12.5))))*(1 / (1 + np.exp(steepness*(np.abs(plot_test_data[:,4]) - 4.0))))*(1 / (1 + np.exp(steepness*(np.abs(plot_test_data[:,5]) - 4.0))))


[[1.         0.         0.06605264 ... 0.         0.         1.        ]
 [1.         0.         0.06605264 ... 0.         0.         1.        ]
 [1.         0.         0.06605209 ... 0.         0.         1.        ]
 ...
 [0.         0.         0.         ... 0.         0.         1.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]
We used PILOWF v0.2 superposition


,timestamp,ws,ws-adjusted
60870,2013-12-11 06:00:00+00:00,0.123522,-0.014797
61136,2013-12-22 08:00:00+00:00,0.057946,-0.002872
38736,2011-06-03 00:00:00+00:00,0.011568,0.011568
24580,2009-10-21 04:00:00+00:00,0.014173,0.014173
46891,2012-05-07 19:00:00+00:00,0.019461,0.019461
...,...,...,...
9412,2008-01-28 04:00:00+00:00,33.932321,33.932321
44278,2012-01-19 22:00:00+00:00,34.273048,34.273048
33424,2010-10-24 16:00:00+00:00,34.325663,34.325663
51883,2012-12-01 19:00:00+00:00,34.421471,34.421472


AssertionError: Testing PILOWF: min ws-adjusted should be non-negative (observed: -0.014797).